In [1]:
import pandas as pd
import re
from tqdm import tqdm
import math
import numpy as np
import json
from ast import literal_eval

In [2]:
df_address = pd.read_excel("address.xlsx", sheet_name = 'ADDRESSES_Song_1非羈縻州縣')
df_address2 = pd.read_excel("address.xlsx", sheet_name = 'ADDRESSES_Tang_Raw')
df_ns=pd.read_csv('../data_dict/cbdb_entity_addresses.csv', sep='\t', header=None).rename(columns={0:'name', 1:'pos'})

In [3]:
# All QSW records.
df_qsw_raw=pd.read_excel('../data_raw/quan_song_wen_muzhi.xlsx', sheet_name='墓誌銘墓表壙誌行狀神道碑塔銘墓碑')[['content_id', 'content', 'subject', 'author']].set_index('content_id')
df_qsw_raw['content']=[str(s) for s in df_qsw_raw['content']]
df_qsw_raw.dropna(inplace=True)
df_qsw_raw = df_qsw_raw[df_qsw_raw['content'] != "nan"]

/usr/remote/anaconda-3.7-deeplearn/lib/python3.7/site-packages/openpyxl/worksheet/_reader.py:300: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [4]:
# Function for seperating sentences.
def sep_mark_sent(string):
    string=string.replace('，', '/w/').replace('；', '/w/').replace('、', '/w/')
    string=string.replace('！', '/w/').replace('。', '/w/').replace('？', '/w/').replace('！', '/w/')
    string=string.replace('：', '/w/')
    return [s for s in string.split("/w/") if s!='']

In [5]:
# Seperate sentences and retain sentences with kinship words.
all_sent_count=0
kin_sent_count=0
df=pd.DataFrame()
for index in tqdm(df_qsw_raw.index):
    content=df_qsw_raw.loc[index]['content']
    sent_list=sep_mark_sent(content) # Seperate sentences.
    all_sent_count+=len(sent_list)
    subject=df_qsw_raw.loc[index]['subject']
    author=df_qsw_raw.loc[index]['author']
    kin_sent_list=[]
    content_id=index
    for sent in sent_list:
        if "葬" in sent or "窆" in sent or "瘞" in sent or "厝" in sent:
            kin_sent_list.append(sent)
    df=pd.concat([pd.DataFrame(data=[[content_id, subject, author, sent, content] for sent in kin_sent_list],
                                          columns=['content_id', 'subject', 'author', 'sent', 'content']
                                          ), df], axis=0, ignore_index=True
                            )

100%|██████████| 4701/4701 [00:04<00:00, 972.95it/s] 


In [6]:
re_rule = "[葬|窆|瘞|厝](.*)"
for index in tqdm(df.index):
    row = df.loc[index]
    find = re.findall(re_rule, row['sent'])[0]
    df.loc[index, 'sent'] = find

100%|██████████| 7650/7650 [00:02<00:00, 3170.22it/s]


In [7]:
re_rule = "(.*[府|州|軍|監])"
for index in tqdm(df.index):
    row = df.loc[index]
    find = re.findall(re_rule, row['sent'])
    if find:
        if "某" not in find[0]:
            find[0] = find[0].replace("於", "")
            find[0] = find[0].replace("于", "")
            find[0] = find[0].replace("君", "")
            find[0] = find[0].replace("公", "")
            find[0] = find[0].replace("之", "")
            find[0] = find[0].replace("其", "")
            if len(find[0]) <= 3 and len(find[0]) > 1 and "其先" not in find[0]:
                df.loc[index, 'first'] = find[0]
    if "河南" in row['sent']:
        df.loc[index, 'first'] = "河南"

100%|██████████| 7650/7650 [00:00<00:00, 8488.68it/s]


In [8]:
df['second'] = [list() for x in range(len(df.index))]
for index in tqdm(df.index):
    row = df.loc[index]
    sent = row['sent']
    sent = sent.replace("于", "")
    sent = sent.replace("縣", "")
    first = df.loc[index, 'first']
    if (str(first) != "nan"):
        sent = sent.replace(first, "")
    for ns in df_ns['name']:
        if ns in sent:
            df.loc[index, 'second'].append(ns)

100%|██████████| 7650/7650 [00:10<00:00, 699.40it/s]


In [9]:
for index in tqdm(df.index):
    row = df.loc[index]
    if "鄉" in row['sent']:
        df.loc[index, 'lang'] = 1
    else:
        df.loc[index, 'lang'] = 0

100%|██████████| 7650/7650 [00:01<00:00, 4107.05it/s]


In [10]:
df_z = pd.DataFrame(columns = ['content_id', 'content', 'sent', 'first' , 'second'])
for index in tqdm(df.index):
    row = df.loc[index]
    first = df.loc[index, 'first']
    second = df.loc[index, 'second']
    if (str(first) != "nan") or len(second) != 0:
        dict = {'content_id':row['content_id'], 'content': row['content'],'sent':row['sent'], 'first':first, 'second':second}
        df_z = df_z.append(dict, ignore_index=True)

100%|██████████| 7650/7650 [00:05<00:00, 1336.44it/s]


In [11]:
df[df['lang'] == 1]

,content_id,subject,author,sent,content,first,second,lang
0,8715497.0,申之,杜士熊,於本縣海隅鄉筦浦之西原,〔額文〕宋故都監杜公墓碑〔碑文〕先君姓杜，諱申之，字伯禄，世居嘉興府華亭之章廟。曾祖顯，故不...,NaN,[海隅],1.0
27,5749291.0,謝達,陳亮,景安於吾邑之合德鄉茅山之原,淳熙三四年間，三山林穎秀實之作邑永康，强敏有幹略，一邑不勞而辦，父老以爲三十年所罕有。劉仲光...,NaN,[景安],1.0
30,5749270.0,陳良能,陳亮,邑之承訓鄉横渡山之東原,往嘗論鄉之富人，以陳性之爲第一。吾友徐元德居厚亦知此翁可人意，而樂妻其少女焉。居厚以對策切直...,NaN,[],1.0
37,5578423.0,柳佾,王汝霖,於岷山廣濟鄉永寧里新兆,宋故知永康軍柳公佾，字公雅，家世秦州成紀縣。曾祖英，贈千牛衛將軍；妣徐氏、劉氏，皆令人。祖淵...,NaN,"[永寧, 廣濟]",1.0
49,5387916.0,楊震,劉一止,公於吴興武康縣崇仁鄉楊村翔鳳山之麓,楊氏望於關西，由漢魏隋唐以至國朝，世胄聯延，列第分坊，博龜襲紫，人物門第之盛，莫與比倫。支分...,NaN,"[興武, 崇仁, 武康, 鳳山]",1.0
...,...,...,...,...,...,...,...,...
7628,4967277.0,江直木,徐鉉,于開封縣汴陽鄉豐臺里,君諱直木，字子建，尋陽人也。廬九之地，時生俊才；忠壯之族，世著南國。末葉湮替，與時汙隆，儒、...,NaN,[開封],1.0
7630,4967269.0,王克貞,徐鉉,于揚州某縣鄉里,君諱克貞，字守節。其先太原人也。緱山之胄，本固源長；淮水既微，枝分派别。所居占籍，吴楚爲多，...,揚州,[],1.0
7632,4967253.0,李從善,徐鉉,于開封府開封縣吹臺鄉蘇里,公諱從善，字子師，隴西成紀人。唐室之諸孫，元和之近屬，譜諜詳悉，此不具陳。若夫天祚之德，大運...,開封府,[開封],1.0
7634,4967204.0,王煜,徐鉉,于河南府某縣某鄉某里,盛德百世，善繼者所以主其祀；聖人無外，善守者不能固其存。蓋運曆之所推，亦古今之一貫。其有享蕃...,河南,[],1.0


In [12]:
df_z.sample(20)

,content_id,content,sent,first,second
191,5631736.0,祖宗時擇儒學爲館職，自館職擇侍從，由侍從擇輔相。所謂儒學者，明仁義禮樂，通古今治亂，其議論可...,武夷鄉上梅里寂歷山中峰僧舍之北,NaN,[武夷]
2014,5216868.0,熙寧、元豐間，士大夫論天下賢者，必曰君實、景仁。其道德風流，足以師表當世；其議論可否，足以榮...,成都之華陽,NaN,"[成都, 華陽]"
2475,5077740.0,吾友何慶之於治平三年五月十九日卒於永泰里第。葬有日，其嗣中行與門人張冕以太常博士田熙叔狀君之...,河南新安,河南,[新安]
142,5750609.0,曾祖士彦，故贈太子少保；妣咸寧郡夫人金氏。 祖畎，贈太子少傅；妣高平郡夫人陳氏。 ...,公于歙縣長壽鄉龍歸里龍山之原,NaN,"[龍山, 長壽]"
2502,5070996.0,公諱志寧，字宗儒，其先渤海蓚人。唐末亂，遠祖避地澤、潞而遷洛，遂爲河南洛陽人。曾祖逵，祖潛，...,於河南縣縣南鄉某原,河南,[南鄉]
2355,5122129.0,嘉祐五年六月辛巳，尚書庫部員外郎、知臨江軍事范君卒於位，年五十有三。其年十月辛酉，葬於江州德...,於江州德化縣之仁貴鄉萬家山前,江州,"[貴鄉, 德化]"
1554,5326410.0,元祐四年七月六日，陳君諱某之妻夫人林氏卒，於是陳君卒八年矣。其葬在興化軍某縣谷山之先塋，其子...,在興化軍某縣谷山之先塋,NaN,"[興化軍, 興化]"
2226,5141023.0,君吴氏，諱某，字某，其先建安大姓。曾大父諱某，建州長史，大父諱某，館驛巡官、檢校尚書吏部員外...,於江州某縣某鄉某原,江州,[]
535,8707897.0,朝散大夫守秘書監知陝州軍府事兼管内勸農使提舉虢商州巡檢兵甲賊盜公事上護軍天水縣開國子食邑...,於河南府密縣義臺鄉劉村之原,河南,[]
2731,5006232.0,昔者騂旄之誓，示天下以懷柔；《麟趾》之詩，明公族之信厚。陶唐之聖，華封陳三者之祝；太姒之賢，...,于開封府開封縣汴陽鄉之禪惠僧舍,開封府,[開封]


In [13]:
df_address = df_address[['c_name_chn', 'y_coord', 'x_coord']]

In [14]:
df_address.drop_duplicates(subset = ['c_name_chn'], inplace=True)

In [15]:
df_address = df_address.set_index('c_name_chn')

In [16]:
df_address2 = df_address2[['c_name_chn', 'y_coord', 'x_coord']]
df_address2.drop_duplicates(subset = ['c_name_chn'], inplace=True)
df_address2 = df_address2.set_index('c_name_chn')

In [17]:
for index in tqdm(df_z.index):
    row = df_z.loc[index]
    first = row['first']
    second = row['second']
    if first in df_address.index:
        df_z.loc[index, "x"] = df_address.loc[first, "x_coord"]
        df_z.loc[index, "y"] = df_address.loc[first, "y_coord"]
    else:
        for place in second:
            if place in df_address.index:
                df_z.loc[index, "x"] = df_address.loc[place, "x_coord"]
                df_z.loc[index, "y"] = df_address.loc[place, "y_coord"]
                break
        if first in df_address2.index:
            df_z.loc[index, "x"] = df_address2.loc[first, "x_coord"]
            df_z.loc[index, "y"] = df_address2.loc[first, "y_coord"]
        else:
            for place in second:
                if place in df_address2.index:
                    df_z.loc[index, "x"] = df_address2.loc[place, "x_coord"]
                    df_z.loc[index, "y"] = df_address2.loc[place, "y_coord"]
                    break

100%|██████████| 2788/2788 [00:01<00:00, 1964.56it/s]


In [18]:
df_z

,content_id,content,sent,first,second,x,y
0,8715497.0,〔額文〕宋故都監杜公墓碑〔碑文〕先君姓杜，諱申之，字伯禄，世居嘉興府華亭之章廟。曾祖顯，故不...,於本縣海隅鄉筦浦之西原,NaN,[海隅],NaN,NaN
1,5749297.0,始余出國北門，彌望沮洳之地而帶以一水，岸行不足以容車馬，湖泊往往隨在而有。舟至松江，風濤汹湧...,于餘杭縣蔡家之塢,NaN,[餘杭],119.94060,30.27177
2,5749291.0,淳熙三四年間，三山林穎秀實之作邑永康，强敏有幹略，一邑不勞而辦，父老以爲三十年所罕有。劉仲光...,景安於吾邑之合德鄉茅山之原,NaN,[景安],NaN,NaN
3,5583141.0,宣和五年，故寶文閣學士連公諱南夫以秘書省校書郎假太常少卿賀女真。來年正月，會金使李靖來告太祖...,于懷安縣稷下里崇福山之原,NaN,"[福山, 懷安]",107.83250,36.64270
4,5578423.0,宋故知永康軍柳公佾，字公雅，家世秦州成紀縣。曾祖英，贈千牛衛將軍；妣徐氏、劉氏，皆令人。祖淵...,於岷山廣濟鄉永寧里新兆,NaN,"[永寧, 廣濟]",111.65340,34.38581
...,...,...,...,...,...,...,...
2783,4967037.0,公諱延禄，字世功。其先上黨人。昔者楚多淫刑，賁始逃難；晋賴謀主，苗受其封。高門之慶，雄視欒、...,于江寧府縣里,江寧府,[],118.76899,32.05256
2784,4967108.0,士有放懷夷曠，介然中立，外物無累于心，没齒不違于道，吾友喬公嘗從事于斯矣。公諱匡舜，字亞元，...,于江寧縣某所,NaN,[江寧],118.76899,32.05256
2785,4967120.0,君諱宣輔，字仲申。其先天水人也，累世從宦，不常厥居。曾祖全真，工部員外郎、滕州刺史。祖倚，太...,江寧府某所,江寧府,[],118.76899,32.05256
2786,4967105.0,君諱某，字某。其先太原人也。昔者諸侯共職，起末運于髭王；儲后上賓，示靈期于瞽史。緱山維岳，肇...,于江都縣某鄉里,NaN,[江都],119.43719,32.39127


In [19]:
df_z = df_z[np.isnan(df_z['x']) == False]

In [20]:
df_z.sample(10)

,content_id,content,sent,first,second,x,y
1233,5517174.0,某從父弟鎬字從周〔二〕。皇曾祖諱璉，皇祖諱愷，俱膏肓泉石。皇考諱汝明，贈右宣教郎。皇妣孺人任...,於吉水縣中鵠鄉白蓮塘之原,NaN,[吉水],115.13225,27.21437
270,5351069.0,惟高氏爲河西大姓，世有顯功，勳在王府，威震夷落。自領軍而下，枝分派别，散居於延安、綏德、...,於霍邑縣永太鄉庫拔村,NaN,[霍邑],111.72808,36.56382
1739,5265420.0,建中靖國元年五月庚辰，觀文殿大學士、太子太保致仕蘇公，薨于潤州。訃聞，上輟視朝三日，贈司空。...,丹徒縣義禮鄉樂安亭五州山之東北阜,NaN,"[丹徒, 樂安]",119.44429,32.20649
2366,5122167.0,公錢氏也，故爲王家，有吴越之地。五世祖鏐，號武肅王。高祖元瓘，文穆王。曾祖儼，昭化軍節度使。...,和州,和州,[],118.36112,31.71422
2457,5093266.0,夫人姓尹氏。父尚書職方郎中、直昭文館，諱少連。郎中有數子，而女獨夫人，又少聰警，識圖書，辨音...,江陰軍江陰縣化龍鄉,江陰軍,[江陰],120.26609,31.90877
41,5947092.0,先君姓王諱應白，字瑞伯，世臨江新淦人。生于嘉定己巳正月丁酉，歿于淳祐癸卯二月乙卯。初娶修德義...,于縣揚名鄉新興里檀木坑之原,NaN,[新興],112.22508,22.70292
2141,5155473.0,季弟青溪縣丞某，字儀父〔一〕。七歲，以先公殁，遺表恩補太廟齋郎。年二十二，調河陰縣主簿，再選...,於東都祥符縣魏陵鄉祖考塋,NaN,[祥符],114.34333,34.78548
1451,5355612.0,政和三年正月，光禄大夫、知樞密院事武昌公抗疏言：‘臣老被病，自度不能復任陛下政事，願賜骸骨歸...,進賢縣麻仙山,NaN,[進賢],116.25907,28.36448
138,5752657.0,公諱章，字叔平，慶元鄞人。曾祖諱轂，左朝奉大夫、知處州，贈光禄大夫，妣永嘉郡夫人葉氏。祖諱灼...,於慈溪縣西嶼鄉東嶺之原,NaN,[慈溪],121.44420,29.97920
2625,5045395.0,清河張公，皇皊四年六月二十九日以疾終於天平之郡寢。明年，其孤壽自宋馳蔡，羸然哭且告予曰：‘我...,魯城,NaN,[魯城],117.17060,38.55884


In [21]:
for i in df_z.index:
    row = df_z.loc[i]
    df_z.loc[i,'sent'] = df_z.loc[i,'sent'].replace("公於", "")
    df_z.loc[i,'sent'] = df_z.loc[i,'sent'].replace("公于", "")
    df_z.loc[i,'sent'] = df_z.loc[i,'sent'].replace("祖考", "")
    df_z.loc[i,'sent'] = df_z.loc[i,'sent'].replace("先考妣", "")
    df_z.loc[i,'sent'] = df_z.loc[i,'sent'].replace("及", "") 
    df_z.loc[i,'sent'] = df_z.loc[i,'sent'].replace("於", "")   
    df_z.loc[i,'sent'] = df_z.loc[i,'sent'].replace("于", "")
    df_z.loc[i,'sent'] = df_z.loc[i,'sent'].replace("君", "")
    df_z.loc[i,'sent'] = df_z.loc[i,'sent'].replace("故", "")
    df_z.loc[i,'sent'] = df_z.loc[i,'sent'].replace("公", "")

/home/jc092/.local/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [22]:
df_z.to_excel("zangdi.xlsx")